In [1]:
import pandas as pd
import csv
import os
import numpy as np
from nltk import pos_tag, word_tokenize
pd.set_option('display.expand_frame_repr', False)


In [2]:
recipes = pd.read_csv('recipes.csv')
recipes.columns=['recipe_id', 'recipe_tag', 'title', 'rating', 'total_reviews', 'ingredients', 'calories', 'carbs',\
                'protein', 'cholesterol', 'sodium']
recipes.head()

,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"[u'1/4 cup olive oil, divided', u'1 onion, dic...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"[u'1 pound spaghetti', u'6 tablespoons olive o...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"[u'2 stalks stalk celery, chopped into 1/4-inc...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"[u'4 cups cold water', u'7/8 cup white sugar',...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"[u'2 cups all-purpose flour', u'1/2 cup cold w...",186,24.1,4.7,7,327


In [ ]:
for element in ingredient:
    print(pos_tag(word_tokenize(element)))


In [ ]:
from nltk.corpus import wordnet as wn
food = wn.synset('food.n.02')
food_ingredients = list(set([w for s in food.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))


food_ingredients

In [3]:
recipes['ingredients'] = recipes['ingredients'].apply(lambda a: a.replace("[", "").replace("]", "").replace("u'","").split("',"))


#.apply(pd.Series).stack()

recipes.head()

,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"[1/4 cup olive oil, divided, 1 onion, diced, ...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"[1 pound spaghetti, 6 tablespoons olive oil, ...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"[2 stalks stalk celery, chopped into 1/4-inch ...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"[4 cups cold water, 7/8 cup white sugar, 2 t...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"[2 cups all-purpose flour, 1/2 cup cold water...",186,24.1,4.7,7,327


In [7]:
def process_ingredients(data):
    data_to_parse = np.asarray(data)
    path = "../ingredient-phrase-tagger/"
    np.savetxt("../ingredient-phrase-tagger/foo.txt", data_to_parse, delimiter=",", fmt='%s')
    # Little bit of a hack to be honest 
    os.system('python ' + path + 'bin/parse-ingredients.py ' + path + 'foo.txt > ' + path + 'other.txt')
    os.system('python ' + path + 'bin/convert-to-json.py ' + path + 'other.txt > ' + "other.json")
    ingredients = pd.read_json('other.json').name.to_frame('ingredients').values
    flat_ingredients = [item for sublist in ingredients for item in sublist]
    flat_ingredients_2 = [item.replace("'","") for item in flat_ingredients]
    return flat_ingredients_2

recipes['ingredients'] = recipes['ingredients'].apply(lambda x: process_ingredients(x))
recipes.head()



,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"[olive oil, onion, all-purpose flour, salt, bl...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"[spaghetti, olive oil, garlic, black pepper, P...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"[celery, olive oil, onion, eggplant, red bell ...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"[water, white sugar, lemon, lemon juice, lemon...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"[all-purpose flour, water, water, olive oil, s...",186,24.1,4.7,7,327


In [10]:
recipes['ingredients'].apply(pd.Series).stack().reset_index(drop=True).to_frame('ingredients')\
.groupby('ingredients')\
.size()\



ingredients
                                         1
(1 inch) piece chopped fresh ginger      1
Alfredo sauce                            1
American cheese                          1
Asafoetida                               1
Asiago cheese                            1
Broth:                                   1
CAKE:                                    1
Caramel:                                 1
Cheddar cheese                          30
Cheddar-Monterey Jack cheese blend       1
Chicken                                  1
Chicken filling:                         1
Chinese cabbage                          1
Colby cheese                             2
Colby-Jack cheese                        1
Colby-Monterey Jack cheese               1
Comte cheese                             1
Custard Base:                            1
Dijon mustard                            4
Dijon-style mustard                      2
Dipping Sauce:                           1
Dough:                                   1

In [ ]:
ingredients = pd.read_json('results.json').name.to_frame('ingredients')
#ingredients.groupby('ingredients').count()
recipe_counts = ingredients.apply(pd.value_counts)
recipe_counts = recipe_counts[recipe_counts['ingredients'] > 5]

recipe_counts
#ingredients.ingredients.value_counts().to_frame().index.values